In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
from src.DataLoader import TableDatasetPath, TableDatasetDF
from functools import partial
from sklearn.utils.class_weight import compute_class_weight

from src.tools import *
from src.trainer import Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 20
NUM_FEATURES = 15

In [3]:
df = pd.read_csv('../data/data_fe.csv')

feature_importance = pd.read_csv('../data/feature_imporstance_WS.csv')

features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df_train, df_test = train_test_split(df,
                                     shuffle=True,
                                     stratify=df['Machine failure'])
# df_train = upsampling(df_train)

In [4]:
train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [5]:
# train_dataset, val_dataset = balance_val_split(dataset)

In [6]:
# class_counts = np.bincount(df['Machine failure'])
# weights = 1.0 / class_counts
# sample_weights = weights[df['Machine failure']]

# # Define sampler with weighted sampling
# sampler = data.WeightedRandomSampler(sample_weights, len(sample_weights))

In [7]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [8]:
model = nn.Sequential(
    nn.Linear(NUM_FEATURES, 500),
    nn.ReLU(),
    nn.Linear(500, 1000),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500, 400),
    nn.ReLU(),
    nn.Linear(400, 300),
    nn.ReLU(),
    nn.Linear(300, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)

In [9]:
model = nn.Sequential(
    nn.Linear(NUM_FEATURES, 256),
    nn.BatchNorm1d(256),
    nn.Dropout(0.3),
    nn.LeakyReLU(0.1),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.Dropout(0.3),
    nn.PReLU(),
    nn.Linear(64, 32),
    nn.BatchNorm1d(32),
    nn.Dropout(0.3),
    nn.GELU(),
    nn.Linear(32, 16),
    nn.BatchNorm1d(16),
    nn.Dropout(0.3),
    nn.ELU(),
    nn.Linear(16, 2)
)


In [10]:
weights_classes = compute_class_weight('balanced',classes = np.unique(df_test['Machine failure']) , y = df_test['Machine failure'])

In [11]:
loss = Loss_class(nn.CrossEntropyLoss(weight=torch.FloatTensor(weights_classes)))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)
optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.9)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=30)

wandb_init_params = dict(
    name='data_fe_lr_1e-3',
    project="Internship_project"
)

In [12]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [13]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:   0%|          | 0/375 [00:00<?, ?it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
I'm studying hard now🧐, don't disturb!:   2%|▏         | 7/375 [00:00<00:05, 68.33it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/

Epoch: 1 of 30, 0.091 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/375 [00:00<?, ?it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
I'm studying hard now🧐, don't disturb!:   3%|▎         | 10/375 [00:00<00:03, 96.81it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib

Epoch: 2 of 30, 0.110 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/375 [00:00<?, ?it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No pos

...

I'm studying hard now🧐, don't disturb!:   0%|          | 0/375 [00:00<?, ?it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No pos

Epoch: 29 of 30, 0.102 min


I'm studying hard now🧐, don't disturb!:   0%|          | 0/375 [00:00<?, ?it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
I'm studying hard now🧐, don't disturb!:   2%|▏         | 9/375 [00:00<00:04, 87.52it/s]/opt/miniconda3/envs/env1/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/opt/miniconda3/envs/env1/lib/

Epoch: 30 of 30, 0.099 min


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
lr,█▇▇▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
test_accuracy,▁▅▇▃▆▃▆▇▇▇▇▅▇▅▇▅▇▇▆█▅▇█▆██▇█▇▇
test_f1_score,▁▃▆▂▄▂▅▆▆▆▆▄▆▄▆▄▆▇▆▇▄▆█▆█▇▆▇▇█
test_loss,█▅▆▅▅▄▄▆▆▅▅▃▄▂▃▂▃▅▂▄▁▃▅▁▁▄▂▂▂▂
train_accuracy,▁▃▄▄▄▅▅▆▅▄▆▄▅▇▆▆▅▇▆▆▇▇█▇▇███▇▇
train_f1_score,▁▃▄▄▄▅▄▅▅▅▅▄▆▆▅▆▅▇▆▇▆▇█▇████▇▇
train_loss,█▇▆▅▅▄▅▄▃▄▄▄▃▃▃▂▃▂▃▂▂▂▁▂▂▂▁▁▂▂
epoch,29
lr,0.00042
test_accuracy,0.9664
